In [14]:
from vespa.package import ApplicationPackage, Schema, Document, Field, RankProfile, HNSW
from vespa.application import Vespa
from vespa.deployment import VespaDocker
import numpy as np
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import time
import pickle
import os
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer, AutoModel
from transformers.onnx.features import FeaturesManager
from transformers.onnx import export
from pathlib import Path

import shutil

In [15]:
# 3. Definição do App

app_package= ApplicationPackage(name="vespasearch1")

app_package.schema.add_fields(
        Field(name="id", type="string", indexing=["summary", "attribute"]),
        Field(name="text", type="string", indexing=["index", "summary"]),
        Field(name="embedding", type="tensor<float>(d[384])", indexing=["attribute", "summary", "index"],
              ann=HNSW(distance_metric="angular")
        #       ann="true",
        #       attribute={"distance-metric": "euclidean"
                                #      , "method": "hnsw"
                                #      }
        ), 
)

app_package.schema.add_rank_profile(
        RankProfile(name="bm25", first_phase="bm25(text)"),
)

app_package.schema.add_rank_profile(
        RankProfile(name="dot_product", first_phase="closeness(embedding)"),
)

app_package.schema.add_rank_profile(
        RankProfile(name="hybrid", first_phase="bm25(text) + closeness(embedding)")
)

In [16]:
output_dir = "./vespa_app"
app_package.to_files(output_dir)

In [17]:
app_package.schema.inputs = [("query_embedding", "tensor<float>(d[384])")]

In [18]:
# 2. Configuração do modelo de embedding (Semantic Search)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 

print("Carregando o modelo de embedding...")

model_id = "sentence-transformers/all-MiniLM-L6-v2"

# Carregue o modelo e tokenizer
model = AutoModel.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Diretório de saída
output = Path("./vespa_app/models/bert")

# Preparar o pipeline
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature="default")
onnx_config = model_onnx_config(model.config)

# Exportar
export(
    preprocessor=tokenizer,
    model=model,
    config=onnx_config,
    opset=17,
    output=output,
)

Carregando o modelo de embedding...


(['input_ids', 'attention_mask', 'token_type_ids'], ['last_hidden_state'])

In [19]:
def add_function_to_schema(schema_path: str, function_name: str, function_expression: str):
    """
    Adiciona uma função ONNX ao arquivo .sd do schema do Vespa.

    Parameters:
    - schema_path: Caminho para o arquivo schema .sd
    - function_name: Nome da função
    - function_expression: Expressão da função (ex: 'onnx("embedding_model", text)')
    """

    # Ler o conteúdo atual do schema
    with open(schema_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Verificar onde inserir: antes do primeiro "rank-profile" ou no final
    insert_index = None
    for i, line in enumerate(lines):
        if line.strip().startswith("rank-profile"):
            insert_index = i
            break

    # Se não achou rank-profile, insere antes do fechamento do schema
    if insert_index is None:
        for i, line in enumerate(lines):
            if line.strip() == "}":
                insert_index = i
                break

    if insert_index is None:
        raise ValueError("Não foi possível localizar onde inserir a função no schema.")

    # Definir a função em formato correto
    function_block = f"""
    function {function_name}() {{
        expression: {function_expression}
    }}
    """

    # Inserir no local identificado
    lines.insert(insert_index, function_block)

    # Salvar o schema modificado
    with open(schema_path, 'w', encoding='utf-8') as f:
        f.writelines(lines)

    print(f"Função '{function_name}' adicionada no schema com sucesso.")

# Caminho para o schema gerado pela API Python:
schema_file = "./vespa_app/schemas/vespasearch1.sd"

# Definir a função
add_function_to_schema(
    schema_path=schema_file,
    function_name="embed_text",
    function_expression='onnx("embedding_model", text)'
)

Função 'embed_text' adicionada no schema com sucesso.


In [20]:
print(app_package.schema.schema_to_text)

schema vespasearch1 {
    document vespasearch1 {
        field id type string {
            indexing: summary | attribute
        }
        field text type string {
            indexing: index | summary
        }
        field embedding type tensor<float>(d[384]) {
            indexing: attribute | summary | index
            attribute {
                distance-metric: angular
            }
            index {
                hnsw {
                    max-links-per-node: 16
                    neighbors-to-explore-at-insert: 200
                }
            }
        }
    }
    rank-profile bm25 {
        first-phase {
            expression {
                bm25(text)
            }
        }
    }
    rank-profile dot_product {
        first-phase {
            expression {
                closeness(embedding)
            }
        }
    }
    rank-profile hybrid {
        first-phase {
            expression {
                bm25(text) + closeness(embedding)
            }
    

In [23]:
# 4. Criando o app e subindo via Docker
# app_package = ApplicationPackage(name="vespasearch1", schema=None)
vespa_docker = VespaDocker(port=8080, container_memory="8G")
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server, 0/60 seconds...
Waiting for configuration server, 5/60 seconds...
Waiting for configuration server, 10/60 seconds...
Waiting for configuration server, 15/60 seconds...
Waiting for configuration server, 20/60 seconds...
Waiting for configuration server, 25/60 seconds...
Waiting for configuration server, 30/60 seconds...
Waiting for configuration server, 35/60 seconds...
Waiting for application to come up, 0/300 seconds.
Waiting for application to come up, 5/300 seconds.
Waiting for application to come up, 10/300 seconds.
Waiting for application to come up, 15/300 seconds.
Waiting for application to come up, 20/300 seconds.
Waiting for application to come up, 25/300 seconds.
Waiting for application to come up, 30/300 seconds.
Application is up!
Finished deployment.


In [24]:
# 5. Indexação de documentos
from tqdm import tqdm
def index_documents(app, docs):
    num_errors = 0
    for doc in tqdm(docs, desc="Indexing documents", unit="document"):
        try:
            doc_id = doc["id"]
            text = doc["text"]
            # embedding = embedding_model.encode(text).tolist()
            app.feed_data_point(
                schema="vespasearch1",
                data_id=doc_id,
                fields={"id": doc_id, "text": text}
            )
        except Exception as e:
            print(f"Error indexing document {doc['id']}: {e}")
            num_errors += 1
            continue
    print(f"Total errors during indexing: {num_errors}")

In [25]:
# 6. Funções de Busca
def lexical_search(app, query):
    return app.query(
        yql="select * from sources * where userQuery();",
        query=query,
        ranking="bm25"
    )

def semantic_search(app, query):
    query_vec = embedding_model.encode(query).tolist()
    return app.query(
        yql="select * from sources * where ([{\"targetNumHits\":10}]nearestNeighbor(embedding, query_embedding));",
        query_tensor={"query_embedding": query_vec},
        ranking="dot_product"
    )

def hybrid_search(app, query):
    query_vec = embedding_model.encode(query).tolist()
    return app.query(
        body={
            "yql": """
                select * from sources * where 
                userQuery() or ([{"targetNumHits":10}]nearestNeighbor(embedding, query_embedding));
            """,
            "query": query,
            "ranking": "hybrid",
            "hits": 10,
            "input.query(query_embedding)": query_vec
        }
    )

In [26]:
PATH = 'subset_msmarco_train_0.01_9.pkl'
PATH_DATA = '../data/' + PATH
PATH_DATA_CLEAN = '../data_clean/' + PATH

In [27]:
# 7 Carregando e formatando dados do MSMARCO
with open(PATH_DATA, "rb") as f:
    data = pickle.load(f)

# Extraindo lista de documentos
docs_raw = data["docs"]
docs = [{"id": doc_id, "text": doc_obj.text} for doc_id, doc_obj in docs_raw.items()]

In [28]:
response = app.get_application_status()
print(response.status_code)
print(response.json())


200
{'application': {'vespa': {'version': '8.357.16'}, 'meta': {'name': 'default', 'user': 'unknown', 'path': '/opt/vespa/var/tmp/deploy14471087300367082730', 'generation': 19, 'timestamp': 1747911214128, 'date': 'Thu May 22 10:53:34 UTC 2025', 'checksum': '362e043f3bbb54884a65f6fa688702cb'}, 'user': {'version': ''}}, 'abstractComponents': [{'id': 'com.yahoo.search.searchers.InputCheckingSearcher@vespasearch1_content', 'class': 'com.yahoo.search.searchers.InputCheckingSearcher', 'bundle': 'container-search-and-docproc:8.357.16'}, {'id': 'com.yahoo.container.logging.FileConnectionLog', 'class': 'com.yahoo.container.logging.FileConnectionLog', 'bundle': 'container-disc:8.357.16'}, {'id': 'com.yahoo.container.jdisc.state.StateMonitor', 'class': 'com.yahoo.container.jdisc.state.StateMonitor', 'bundle': 'container-disc:8.357.16'}, {'id': 'com.yahoo.search.searchers.OpportunisticWeakAndSearcher@vespa', 'class': 'com.yahoo.search.searchers.OpportunisticWeakAndSearcher', 'bundle': 'container-s

In [29]:
import requests

r = requests.get("http://localhost:8080/document/v1/document/document/docid/0")
print(r.status_code, r.text)

500 {"pathId":"/document/v1/document/document/docid/0","id":"id:document:document::0","message":"[UNKNOWN(251012) @ tcp/vespasearch1:19112/default]: Unknown bucket space mapping for document type 'document' in id: 'id:document:document::0' "}


In [30]:
import os
print("Local esperado:", os.path.abspath("schemas/document.sd"))
print("Existe?", os.path.exists("schemas/document.sd"))

with open("schemas/document.sd", "r") as f:
    print("Conteúdo:\n")
    print(f.read())

Local esperado: c:\Users\lfama\Documentos\FGV\7º Período\data-sci-project\src\schemas\document.sd
Existe? True
Conteúdo:

schema document {
    document document {
        field id type string {
            indexing: summary | attribute
        }

        field text type string {
            indexing: index | summary
        }

        field embedding type tensor<float>(d[384]) {
            indexing: attribute | summary
            attribute {
                distance-metric: dotproduct
            }
        }
    }

    rank-profile bm25 inherits default {
        first-phase {
            expression: nativeRank(text)
        }
    }

    rank-profile dot_product inherits default {
        first-phase {
            expression: closeness(embedding)
        }
    }

    rank-profile hybrid inherits default {
        first-phase {
            expression: nativeRank(text) + closeness(embedding)
        }
    }
}


In [31]:
index_documents(app, docs)

Indexing documents:  19%|█▉        | 5216/27778 [06:11<87:15:40, 13.92s/document] 

Error indexing document msmarco_passage_03_394865952: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_03_395488422: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_03_395537598: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Indexing documents:  19%|█▉        | 5220/27778 [06:11<33:45:44,  5.39s/document]

Error indexing document msmarco_passage_03_395556013: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_03_395804315: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_03_395904835: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_03_396826350: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_03_397127365: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_03_397263259: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Indexing documents:  19%|█▉        | 5226/27778 [06:11<11:50:26,  1.89s/document]

...

Error indexing document msmarco_passage_02_864924762: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_864969977: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_865047722: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_865074436: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_865622571: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_865670651: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_865791930: ('Connection aborted.', RemoteDisconnected('Remote end closed co

Indexing documents: 100%|█████████▉| 27765/27778 [38:46<00:00, 42.74document/s]

Error indexing document msmarco_passage_02_866261955: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_866269386: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_866279090: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_866401766: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_866773973: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_866961537: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_867349876: ('Connection aborted.', RemoteDisconnected('Remote end closed co

Indexing documents: 100%|██████████| 27778/27778 [38:46<00:00, 11.94document/s]

Error indexing document msmarco_passage_02_867705229: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_867723418: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_867931258: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_868066639: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_868104434: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_868243546: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error indexing document msmarco_passage_02_868510218: ('Connection aborted.', RemoteDisconnected('Remote end closed co

In [32]:
result.json

NameError: name 'result' is not defined

In [ ]:
response = app.query(
    body={
        "yql": "select * from sources * where true;",
        "hits": 10
    }, timeout = 1000
)

for hit in response.hits:
    print(hit["fields"])


{'sddocname': 'vespasearch1', 'documentid': 'id:vespasearch1:vespasearch1::msmarco_passage_31_15605486', 'id': 'msmarco_passage_31_15605486', 'text': 'Summary. Factor VII deficiency is a rare genetic bleeding disorder characterized by a deficiency or reduced activity of clotting factor VII. Clotting factors are specialized proteins that are essential for the blood to clot normally. Individuals with factor VII deficiency can experience prolonged, uncontrolled bleeding episodes.', 'embedding': {'type': 'tensor<float>(d[384])', 'values': [-0.02649758942425251, -0.04880521073937416, -0.037282705307006836, 0.04838443547487259, 0.07315978407859802, 0.05858995392918587, -0.04715218022465706, 0.08747385442256927, -0.0536402091383934, 0.0022039851173758507, -0.044738803058862686, -0.04408184811472893, -0.054344192147254944, 0.056642740964889526, -0.09955909848213196, -0.06986312568187714, -0.016022592782974243, 0.03696442395448685, -0.08723299205303192, -0.036453329026699066, -0.076594926416873

In [ ]:
response = app.query(
    body={
        "yql": "select * from sources * where userQuery();",
        "query": "If my name is Ramyro, what is the capital of France?",
        "hits": 5,
        "ranking": "bm25"
    }, timeout = 10000
)
print(response.hits)

[]


In [ ]:
hybrid_search(app, "What is the capital of France?")


VespaError: [{'code': 12, 'summary': 'Timed out', 'source': 'vespasearch1_content', 'message': "Error in execution of chain 'vespasearch1_content': Chain timed out."}]